### ЗАДАЧА: Контроль доставок и просрочек

Логистическая компания ведёт учёт отправок по складам.
Нужно обработать файл с доставками, найти просрочки, дубликаты и сформировать отчёт для менеджера.

НЕОБХОДИМО РЕАЛИЗОВАТЬ:

1. Сохранить исходный список строк в файл delivery_log.txt.

2. Прочитать файл delivery_log.txt через with open и загрузить записи в список словарей.

3. Для каждого склада посчитать:
   - общее количество отправок
   - количество просроченных отправок (actual_day > planned_day).

4. Собрать множество всех городов доставки.

5. Найти shipment_id, которые встречаются больше одного раза (дубликаты).

6. Для каждого города посчитать суммарное количество доставленных единиц товара (только status = delivered).

7. Найти склад с самым высоким процентом просрочек.

8. Записать итоговый отчёт в файл delivery_report.txt (статистика складов, города, дубликаты, топ-склад по просрочкам, объёмы по городам).

In [ ]:
lines = [
    "SHP-1001,North,Chicago,3,3,40,delivered",
    "SHP-1002,North,Boston,4,6,20,delivered",
    "SHP-1003,East,Miami,2,2,35,cancelled",
    "SHP-1004,East,Chicago,5,8,15,delivered",
    "SHP-1005,West,Dallas,1,1,50,delivered",
    "SHP-1006,West,Miami,2,4,30,delivered",
    "SHP-1002,North,Boston,4,6,20,delivered",
    "SHP-1007,South,Atlanta,6,6,25,in_transit",
    "SHP-1008,South,Dallas,3,5,10,delivered"
]

deliveries = []
# список словарей с записями доставок
# ключи: shipment_id, warehouse, city, planned_day, actual_day, items, status

warehouse_stats = {}
# warehouse -> {"total": 0, "delayed": 0}

cities = set()
# множество городов доставки

shipment_counter = {}
# shipment_id -> количество встреч

duplicate_shipments = []
# список shipment_id, которые дублируются

city_delivered_items = {}
# city -> суммарное количество items для status == delivered


with open("delivery_log.txt", "w", encoding="utf-8") as file:
    # записать строки lines в файл
    for line in lines:
        file.write(line + "\n")


with open("delivery_log.txt", "r", encoding="utf-8") as file:
    for line in file:
        line = line.strip()
        if not line:
            continue

        parts = line.split(",")     #  разбить строку по ','
        shipment_id, warehouse, city = parts[0], parts[1], parts[2]   
        planned_day = int(parts[3])           # planned_day, actual_day, items перевести в int
        actual_day = int(parts[4])
        items = int(parts[5])
        status = parts[6]
        deliveries.append({            # собрать словарь и добавить его в deliveries

            "shipment_id": shipment_id,
            "warehouse": warehouse,
            "city": city,
            "planned_day": planned_day,
            "actual_day": actual_day,
            "items": items,
            "status": status
        })   

for delivery in deliveries:
    # получить shipment_id, warehouse, city, planned_day, actual_day, items, status
    shipment_id = delivery["shipment_id"]
    warehouse = delivery["warehouse"]
    city = delivery["city"]
    planned_day = delivery["planned_day"]
    actual_day = delivery["actual_day"]
    items = delivery["items"]
    status = delivery["status"]

    if warehouse not in warehouse_stats:   # обновить warehouse_stats
        warehouse_stats[warehouse] = {"total": 0, "delayed": 0}
    warehouse_stats[warehouse]["total"] += 1

    if actual_day > planned_day:   #  если actual_day > planned_day, увеличить delayed
        warehouse_stats[warehouse]["delayed"] += 1

    cities.add(city)   #  добавить city в cities

    shipment_counter[city] = shipment_counter.get(city, 0) + 1   #  обновить shipment_counter

    if status == "delivered":   # если status == delivered, обновить city_delivered_items
        city_delivered_items[status] = city_delivered_items.get(status, 0) + 1


for shipment_id, count in shipment_counter.items():
    # если count > 1, добавить shipment_id в duplicate_shipments
    if count > 1:
        duplicate_shipments.append(shipment_id)

worst_warehouse = None
max_delay_rate = -1

for warehouse, stats in warehouse_stats.items():
    total = stats["total"]
    delayed = stats["delayed"]
    delay_rate = delayed / total
    if delay_rate > max_delay_rate:
        max_delay_rate = delay_rate
        worst_warehouse = warehouse
# пройти по warehouse_stats
# для каждого склада посчитать delay_rate = delayed / total
# найти склад с максимальным delay_rate


with open("delivery_report.txt", "w", encoding="utf-8") as file:
    # записать в отчет:
    # - статистику по складам
    # - список городов
    # - дубликаты shipment_id
    # - склад с максимальным процентом просрочек
    # - доставленные объёмы по городам
    file.write("Статистику по складам: \n")
    for warehouse, stats in warehouse_stats.items():
        file.write(f"{warehouse}: всего отгружено - {stats["total"]}, просроченные отгрузки - {stats["delayed"]}\n")

    file.write("список городов:\n")
    file.write(",".join(cities) + "\n")

    file.write("Дубликаты shipment_id: ")
    if duplicate_shipments:
        file.write(",".join(duplicate_shipments) + "\n")
    else:
        file.write("дубликатов нет\n")

    file.write(f"Склад с максимальным процентом просрочек: {worst_warehouse} ({max_delay_rate})\n")

    file.write("Доставленные объемы по городам: \n")
    for city, total_items in city_delivered_items.items():
        file.write(f"{city}: {total_items}\n")
